In [1]:
import torch

from tqdm import tqdm

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [ ]:
from pde.models.conv_lstm import ConvLSTM_Model
from pde.train_utils.dataloader_openstl import load_data
from pde.train_utils.losses import calculate_pde_and_continuity_loss, weighted_mae_torch

In [4]:
data_root = "/home/fa.buzaev/data_5/"
last_hour_in = 13
last_hour_out = 25
exp_root = 'exp_convlstm'
model_root = f"/home/asutkin/kursach/{exp_root}/best_models/model_mistake.pt"

In [14]:
dataloader_train, dataloader_vali, dataloader_test, mean, std = load_data(batch_size=12,
                                                                          val_batch_size=12,
                                                                          data_root=data_root,
                                                                          num_workers=6,
                                                                          # data_split='2_8125',
                                                                          # data_name='mv3',
                                                                          data_split='5_625',
                                                                          data_name='uv10',
                                                                          # data_name='mv6',
                                                                          # train_time=['1979', '2015'],
                                                                          train_time=['2010', '2011'],
                                                                          val_time=['2016', '2016'],
                                                                          test_time=['2017', '2018'],
                                                                          idx_in=[*range(1, last_hour_in)],
                                                                          idx_out=[*range(last_hour_in + 12, last_hour_out + 12)],
                                                                          step=1,
                                                                          level=1,
                                                                          distributed=False, use_augment=False, 
                                                                          use_prefetcher=False, drop_last=False)

/home/asutkin/.conda/envs/vrwkv/lib/python3.11/site-packages/xarray/core/accessor_dt.py:441: FutureWarning: dt.weekofyear and dt.week have been deprecated. Please use dt.isocalendar().week instead.
  warnings.warn(
/home/asutkin/.conda/envs/vrwkv/lib/python3.11/site-packages/xarray/core/accessor_dt.py:441: FutureWarning: dt.weekofyear and dt.week have been deprecated. Please use dt.isocalendar().week instead.
  warnings.warn(
/home/asutkin/.conda/envs/vrwkv/lib/python3.11/site-packages/xarray/core/accessor_dt.py:441: FutureWarning: dt.weekofyear and dt.week have been deprecated. Please use dt.isocalendar().week instead.
  warnings.warn(
/home/asutkin/.conda/envs/vrwkv/lib/python3.11/site-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 1, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoade

In [ ]:
from tqdm import tqdm
import random
import json
import sys, os

criterion = torch.nn.L1Loss()
configs = {
    'in_shape': [12, 2, 32, 64],
    'filter_size': 5,
    'patch_size': 1,
    'stride': 1,
    'pre_seq_length': 12,
    'aft_seq_length': 1,
    'layer_norm': 0,
    
}
model = ConvLSTM_Model(1, [128, 128, 128, 128], configs).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)
checkpoint = torch.load(model_root)
model.load_state_dict(checkpoint['model'])
optimizer.load_state_dict(checkpoint['optimizer'])
best_loss = checkpoint['loss']
# best_loss=float('inf')
with open(f"/home/asutkin/kursach/{exp_root}/logs_mistake.txt", 'a') as file:
    for i in range(4, 5):
        model.train()
        train_loss = 0

        for x_train, y_train in tqdm(dataloader_train, desc='Training'):
            x_train, y_train = x_train.to(device), y_train.to(device)
            optimizer.zero_grad()
            big_loss = 0
            for j in range(1, last_hour_out - last_hour_in):
                time_tensor = j * torch.ones(x_train.shape[0], device=x_train.device).unsqueeze(-1)
                prediction = model(x_train, time_tensor)
                loss = criterion(prediction[:,0,:,:,:], y_train[:,j,:,:,:])
                x_train = torch.cat((x_train[:,1:,:,:,:], prediction[:,0,:,:,:].unsqueeze(1)), dim=1)
                big_loss += loss
            big_loss.backward()
            optimizer.step()
            train_loss += big_loss.item() / (last_hour_out - last_hour_in)
            # torch.cuda.empty_cache()
        train_loss /= len(dataloader_train)

        model.eval()
        val_loss = 0

        for x_val, y_val in tqdm(dataloader_vali, desc='Validating'):
            x_val, y_val = x_val.to(device), y_val.to(device)
            big_loss_val = 0
            for j in range(1, last_hour_out - last_hour_in):
                time_tensor = j * torch.ones(x_val.shape, device=x_val.device).unsqueeze(-1)
                prediction = model(x_val, time_tensor)
                loss = criterion(prediction[:,0,:,:,:], y_val[:,j,:,:,:])
                x_val = torch.cat((x_val[:,1:,:,:,:], prediction[:,0,:,:,:].unsqueeze(1)), dim=1)
                big_loss_val += loss
            val_loss += big_loss_val.item() / (last_hour_out - last_hour_in)
        val_loss /= len(dataloader_vali)

        if val_loss < best_loss:
            best_loss = val_loss
            torch.save({
                'model': model.state_dict(),
                'optimizer': optimizer.state_dict(),
                'loss': best_loss,
            }, model_root)

        print(f"Epoch: {i + 1}; Train_loss: {train_loss}; Vall_loss: {val_loss}")
        log = json.dumps({'Epoch': i+1, 'Train_loss': train_loss, 'Val_loss': val_loss})
        file.write(f"{log}\n")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import clear_output
from tqdm import tqdm

def denorm(item, std, mean, idx=0):
    mean = mean.squeeze()[idx]
    std = std.squeeze()[idx]
    item_denorm = item * std + mean
    return item_denorm

for x_test, y_test in dataloader_test:
    x_test, y_test = x_test.to(device), y_test.to(device)
    break
    
x_test_ = torch.empty(x_test.shape, device=device)

configs = {
    'in_shape': [12, 2, 32, 64],
    'filter_size': 5,
    'patch_size': 1,
    'stride': 1,
    'pre_seq_length': 12,
    'aft_seq_length': 1,
    'layer_norm': 0,
    
}
model = ConvLSTM_Model(1, [128, 128, 128, 128], configs)
criterion = torch.nn.MSELoss()
checkpoint = torch.load(model_root)
model.load_state_dict(checkpoint['model']).to(device)
model.eval()
t = 0
x_data = x_test

for i in tqdm(range(1, 73)):
    time_tensor = i * torch.ones(x_test.shape[0], device=x_test.device).unsqueeze(-1)
    with torch.no_grad():
        prediction = model(x_data, time_tensor)
    plt.figure(constrained_layout=True, figsize=(32, 6))
    x_data = torch.cat((x_data[:,1:,:,:,:], prediction.unsqueeze(1)), dim=1)
    
    plt.subplot(131)
    plt.imshow(denorm(prediction[7, 0], std, mean).squeeze().detach().cpu().numpy())
    plt.title(f"Prediction U wind by Andrew, step={t}")
    # plt.colorbar()

    plt.subplot(132)
    plt.imshow(denorm(y_test[7, t, 0], std, mean).squeeze().detach().cpu().numpy())
    plt.title(f"True answer, step={t}")
    plt.colorbar(boundaries=np.linspace(-20, 20, 20)) 

    plt.subplot(133)
    plt.imshow(np.abs(denorm(prediction[7, 0], std, mean).squeeze().detach().cpu().numpy() - denorm(y_test[7, t, 0], std, mean).squeeze().detach().cpu().numpy()))
    plt.title(f"Absolute difference, step={t}")
    plt.colorbar(boundaries=np.linspace(0, 20, 20))
    clear_output(wait=True)

    plt.savefig(f"/home/asutkin/kursach/{exp_root}/predictions/imvp_{t}.png")

    t += 1

In [ ]:
import imageio

images = []
for i in range(0,72):
    images.append(imageio.imread(f"/home/asutkin/kursach/{exp_root}/predictions/imvp_{i}.png"))


imageio.mimsave(f"/home/asutkin/kursach/{exp_root}/predictions/wind_norm.gif", images)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(constrained_layout=True, figsize=(32, 6))

plt.imshow(prediction[65, 0].squeeze().detach().cpu().numpy())

In [ ]:
plt.figure(constrained_layout=True, figsize=(32, 6))
plt.imshow(y_test[65, 1, 0].squeeze().detach().cpu().numpy())

In [ ]:
import numpy as np
plt.figure(constrained_layout=True, figsize=(32, 6))
plt.imshow(np.abs(y_test[65, 0, 0].squeeze().detach().cpu().numpy() - prediction[65, 0].squeeze().detach().cpu().numpy()))
plt.colorbar(boundaries=np.linspace(-20, 20, 20)) 

In [14]:
torch.save({
    'model': model.state_dict(),
    'optimizer': optimizer.state_dict(),
}, '/home/asutkin/kursach/model.pt')

In [15]:
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import clear_output
from tqdm import tqdm
from pde.train_utils.losses import calculate_pde_and_continuity_loss, weighted_rmse_torch

def denorm(item, std, mean, idx=0):
    mean = mean.squeeze()[idx]
    std = std.squeeze()[idx]
    item_denorm = item * std + mean
    return item_denorm

configs = {
    'in_shape': [12, 2, 32, 64],
    'filter_size': 5,
    'patch_size': 1,
    'stride': 1,
    'pre_seq_length': 12,
    'aft_seq_length': 1,
    'layer_norm': 0,
    
}
model = ConvLSTM_Model(1, [128, 128, 128, 128], configs).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)
checkpoint = torch.load(model_root)
model.load_state_dict(checkpoint['model'])
model.eval()
for x_test, y_test in tqdm(dataloader_test):
    x_test, y_test = x_test.to(device), y_test.to(device)
    t = 0
    x_data = x_test
    array_to_plot = np.empty((last_hour_out - last_hour_in))
    for j in range(1, last_hour_out - last_hour_in):
        time_tensor = j * torch.ones(x_test.shape[0], device=x_test.device).unsqueeze(-1)
        prediction = model(x_test, time_tensor)
        x_test = torch.cat((x_test[:,1:,:,:,:], prediction[:,0,:,:,:].unsqueeze(1)), dim=1)

        u_wind_prediction = denorm(prediction[:,0,0,:,:], std, mean)
        metric_u_wind = weighted_rmse_torch(u_wind_prediction.unsqueeze(1), denorm(y_test[:, t, 0], std, mean).unsqueeze(1))

        v_wind_prediction = denorm(prediction[:,0,1,:,:], std, mean, idx=1)
        metric_v_wind = weighted_rmse_torch(v_wind_prediction.unsqueeze(1), denorm(y_test[:, t, 1], std, mean, idx=1).unsqueeze(1))

        metric = (metric_u_wind + metric_v_wind) / 2

        array_to_plot[t] = metric.detach().cpu().item()
        t += 1
    
    all_batches.append(array_to_plot)


100%|██████████| 1457/1457 [09:32<00:00,  2.55it/s]
